In [1]:
%matplotlib inline

import sys, os
sys.path.append(os.path.abspath("..")) # add parent directory for import

## Chapter 3 : Using The Grid

In the previous chapter we used itcsimlib to identify a combination of model parameters that optimized a fit to three sets of experimental data.

However, just as important as finding best-fit parameter values is estimating the confidence intervals for those values. Itcsimlib can do this through several ways, but the simplest is bootstrapped (also known as jackknife) uncertainty estimation.

As usual, set up the simulator, load the experimental datasets we used in chapter two, and initialize the `ITCFit` object:

In [2]:
from itcsimlib import *
from itcsimlib.model_independent import *

sim = ITCSim(T0=298.15, units="kcal", verbose=False)
sim.set_model( OneMode() )
sim.add_experiment_file('chapter_2_example_288K.txt', skip=[0])
sim.add_experiment_file('chapter_2_example_298K.txt', skip=[0])
sim.add_experiment_file('chapter_2_example_308K.txt', skip=[0])

fit = ITCFit( sim, verbose=False )

Next, retrieve and apply the optimized parameters we generated in the previous chapter:

In [3]:
from itcsimlib.utilities import read_params_from_file

tmp = read_params_from_file( "chapter_2_parameters.txt", row=1 )
sim.set_model_params( **tmp )

We use the "estimate" method of itcsimlib's `ITCFit` object to evaluate the bootstrapped datasets.

For the sake of time, we'll estimate the variance in just the $\Delta C_{p}$ parameter using 20 bootstraps. Note that in reality, we'd want to run at least several hundred, and we'd want to let all of the model parameters float.

In [4]:
params = fit.estimate( params=['dCp'], bootstraps=20 )

The `params` dictionary object contains the average value and standard deviation of the `dCp` ($\Delta C_{p}$) parameter, which we can either save to a file or print to the console for our information:

In [5]:
print("dCp Mean: %.3f kcal +/- %.3f kcal"%params['dCp'])

dCp Mean: -0.126 kcal +/- 0.003 kcal


Sometimes it's useful to systematically vary one or more parameters, such as sampling different starting conditions for fitting, or to get a better understanding of how model parameter are correlated.

Let's see what happens when we systematically vary the stoichiometry of our model from 0.5 to 2.5 ligands per macromolecule. Itcsimlib makes this easy through the `ITCGrid` object, and to which we pass our fit object that we want to use:

In [6]:
grid = ITCGrid( fit, verbose=False )

Add an axis for our grid. In this case, our grid is only one dimension (different stoichiometry values, so it only has one axis).

Also, let's evaluate our $\Delta C_{p}$ range over 11 linearly-sampled steps:

In [7]:
grid.add_axis( param='n', start=0.5, stop=2.5, steps=11, logspace=False )

Now, we optimize our parameters at each grid point.

First, let's use our range of stochiometry values simply as starting points (i.e. not held to the grid value). We'll call `grid.optimize()` like this:

In [8]:
results = grid.optimize( params=['n','dG','dH','dCp'] )

We can iterate over the results provided by the grid-sampled starting conditions and print out a nicely formatted list: 

In [9]:
for gridpt,v,x in results:
    print("n=%.1f, chisq=%.2f: n=%.2f, dG=%.2f, dH=%.2f, dCp=%.2f"%(gridpt[0], x, v['n'], v['dG'], v['dH'], v['dCp']))

n=0.5, chisq=0.49: n=1.80, dG=-10.89, dH=-11.75, dCp=-0.13
n=0.7, chisq=0.49: n=1.80, dG=-10.89, dH=-11.76, dCp=-0.13
n=0.9, chisq=0.49: n=1.80, dG=-10.89, dH=-11.75, dCp=-0.13
n=1.1, chisq=0.49: n=1.80, dG=-10.89, dH=-11.76, dCp=-0.13
n=1.3, chisq=0.49: n=1.80, dG=-10.89, dH=-11.76, dCp=-0.13
n=1.5, chisq=0.49: n=1.80, dG=-10.89, dH=-11.76, dCp=-0.13
n=1.7, chisq=0.49: n=1.80, dG=-10.89, dH=-11.75, dCp=-0.13
n=1.9, chisq=0.49: n=1.80, dG=-10.89, dH=-11.75, dCp=-0.13
n=2.1, chisq=0.49: n=1.80, dG=-10.89, dH=-11.76, dCp=-0.13
n=2.3, chisq=0.49: n=1.80, dG=-10.89, dH=-11.76, dCp=-0.13
n=2.5, chisq=0.49: n=1.80, dG=-10.89, dH=-11.75, dCp=-0.13


You should notice from this grid search that regardless of the starting stoichiometry, fits always converge to around n = 1.8. Now, let's hold the stoichiometry fixed at each pre-set grid point, and only optimize the other parameters.

This is done by simply leaving out the `n` parameter from the optimization parameters:

In [10]:
results = grid.optimize( params=['dG','dH','dCp'] )

for gridpt,v,x in results:
    print("n=%.1f, chisq=%.2f: dG=%.2f, dH=%.2f, dCp=%.2f"%(gridpt[0], x, v['dG'], v['dH'], v['dCp']))   

n=0.5, chisq=253.76: dG=-9.33, dH=-30.80, dCp=-1.75
n=0.7, chisq=176.41: dG=-9.16, dH=-27.29, dCp=-1.34
n=0.9, chisq=123.47: dG=-9.09, dH=-24.10, dCp=-1.01
n=1.1, chisq=86.76: dG=-9.12, dH=-20.74, dCp=-0.70
n=1.3, chisq=58.13: dG=-9.40, dH=-16.98, dCp=-0.41
n=1.5, chisq=28.86: dG=-10.05, dH=-13.82, dCp=-0.21
n=1.7, chisq=4.51: dG=-10.73, dH=-12.23, dCp=-0.14
n=1.9, chisq=3.96: dG=-10.86, dH=-11.42, dCp=-0.12
n=2.1, chisq=26.44: dG=-10.48, dH=-10.92, dCp=-0.13
n=2.3, chisq=56.53: dG=-10.07, dH=-10.53, dCp=-0.14
n=2.5, chisq=86.51: dG=-9.71, dH=-10.27, dCp=-0.15


You will notice that the $\chi^{2}$ values for stoichiometries far from n=1.8 are high, but become much smaller as the fixed stoichiometry approaches 1.8

That concludes this chapter, in chapter 4 we will cover developing and using custom models.

In [11]:
sim.done()